In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import urllib.request, json

In [2]:
#   NS Bron aanpassen
# Inladen CSV en toevoegen kolommen voor Stoelen en Bezette stoelen
TravelData = pd.read_csv('OC_NS_20240920.csv')
TravelData['Seats'] = 0
TravelData['Occupied Seats'] = 0

# Voor iedere treinserie het aantal stoelen toevoegen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'VehicleType'] == "VIRM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "DDZ":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "FLIRT FFF":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 53
    if TravelData.loc[i, 'VehicleType'] == "ICM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 75
    if TravelData.loc[i, 'VehicleType'] == "ICNG25":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 52
    if TravelData.loc[i, 'VehicleType'] == "SLT":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 54
    if TravelData.loc[i, 'VehicleType'] == "SNG":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 50
    if TravelData.loc[i, 'VehicleType'] == "SW7-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48
    if TravelData.loc[i, 'VehicleType'] == "SW9-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48

# Maar 1 week behouden dus 3 dagen weghalen uit de tabel
TravelData['OperatingDay'] = pd.to_datetime(TravelData['OperatingDay'])
dates_to_exclude = pd.to_datetime(['2024-09-20', '2024-09-21', '2024-09-22'])
df_filtered = TravelData[~TravelData['OperatingDay'].isin(dates_to_exclude)]
TravelData = df_filtered.reset_index(drop=True)

#treincodes boven 700000 aanpassen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'JourneyNumber'] > 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 700000

for i in range(len(TravelData)):
    if 200000 < TravelData.loc[i, 'JourneyNumber'] < 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 200000

display(TravelData)

,DataOwnerCode,OperatingDay,LinePlanningNumber,JourneyNumber,ReinforcementNumber,TimingLinkOrder,UserStopCodeBegin,UserStopCodeEnd,Occupancy,VehicleType,TotalNumberOfCoaches,Seats,Occupied Seats
0,NS,2024-09-29,NaN,6430,0,2,EHS,BET,2,FLIRT FFF,3,159,0
1,NS,2024-09-23,NaN,3328,0,2,LAA,GVM,2,SNG,8,400,0
2,NS,2024-09-23,NaN,3626,0,8,TB,HT,3,VIRM,8,800,0
3,NS,2024-09-26,NaN,7039,0,5,DVC,HON,2,SNG,3,150,0
4,NS,2024-09-29,NaN,5834,0,16,SHL,HFD,2,SLT,6,324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209204,NS,2024-09-25,NaN,1824,0,6,LLS,ALM,3,DDZ,6,600,0
209205,NS,2024-09-24,NaN,6950,0,1,UT,UTLR,4,SLT,4,216,0
209206,NS,2024-09-29,NaN,5117,0,4,RSW,DT,2,SNG,4,200,0
209207,NS,2024-09-28,NaN,7078,0,3,APDO,APD,2,SNG,3,150,0


In [3]:
#   Voor iedere treinserie een dataframe maken
#Inladen CSV met alle NS treindiensten
TrainServices = pd.read_csv('TrainServices.csv', delimiter = ';')

#Dictionary maken met per treinserie een dataframe met ieder stukje tussen stations er in
dataframes_dict = {}

TrainServices.loc[0,'Code']
TrainServices.loc[0,'String'].split(',')

for i in range(len(TrainServices)):
    a = TrainServices.loc[i,'String'].split(',')
    df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
    for j in range(len(a) - 1):
        new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
        df = pd.concat([df, new_row], ignore_index=True)
    dataframes_dict[TrainServices.loc[i,'Code']] = df

In [5]:
for i in range(len(TravelData)):
    a = TravelData.loc[i, 'JourneyNumber']//100
    #print(a)
    c = 0
    for j in range(len(dataframes_dict[a*100])):
        if TravelData.loc[i, 'UserStopCodeBegin'] == dataframes_dict[a*100].loc[j, 'From'].upper():
            c = 1
        if c == 1:
            if dataframes_dict[a*100].loc[j, 'Seats'] == None:
                dataframes_dict[a*100].loc[j, 'Seats'] = TravelData.loc[i, 'Seats']
            else:
                dataframes_dict[a*100].loc[j, 'Seats'] += TravelData.loc[i, 'Seats']
        if TravelData.loc[i, 'UserStopCodeEnd'] == dataframes_dict[a*100].loc[j, 'To'].upper():
            break

In [8]:
print(dataframes_dict[6000])
display(dataframes_dict)
#print(TravelData.loc[20, 'Seats'])
#print(dataframes_dict)

   From    To   Seats
0    Ut  Utvr   95148
1  Utvr   Utl  188892
2   Utl   Htn  188892
3   Htn  Htnc  377784
4  Htnc    Cl  377784
5    Cl   Gdm  377784
6   Gdm   Zbm  377784
7   Zbm    Ht  377784


{np.int64(500):     From    To   Seats
 0     Gn  Gerp   53700
 1   Gerp   Hrn   56800
 2    Hrn   Asn   56800
 3    Asn    Bl  110550
 4     Bl   Hgv  110550
 5    Hgv    Mp  110550
 6     Mp    Zl  112075
 7     Zl    Wz  119375
 8     Wz   Hde  119375
 9    Hde    Ns  119375
 10    Ns    Hd  119375
 11    Hd   Eml  119375
 12   Eml    Pt  119375
 13    Pt   Nkk  119375
 14   Nkk  Avat  119375
 15  Avat  Amfs  119375
 16  Amfs   Amf  119375
 17   Amf   Dld  110750
 18   Dld   Bhv  110750
 19   Bhv   Uto  110750
 20   Uto    Ut  110750
 21    Ut  Utlr  115600
 22  Utlr   Utt  115600
 23   Utt   Vtn  115600
 24   Vtn    Wd  115600
 25    Wd   Gdg  115600
 26   Gdg    Gd  115600
 27    Gd   Nwk  115600
 28   Nwk   Cps  115600
 29   Cps   Rta  115600
 30   Rta   Rtn  114475
 31   Rtn   Rtd  114475,
 np.int64(600):     From    To   Seats
 0     Lw    Gw   49725
 1     Gw   Akm   49725
 2    Akm    Hr   50125
 3     Hr    Wv  125225
 4     Wv   Swk  125225
 5    Swk    Mp  125225
 6     Mp